# Music track analysis using Librosa

### First we install. Pip did not work, but conda with the "-y" flag for silent install worked

In [6]:
#!conda install -y -c conda-forge librosa

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



### Importing libraries

In [37]:
#from __future__ import print_function
import librosa
import numpy as np

### Now we get a sample audio file

We could have used tracks on S3 also

In [8]:
# Get the file path to the included audio example
filename = librosa.util.example_audio_file()
print("filename: ", filename)

filename:  /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/librosa/util/example_data/Kevin_MacLeod_-_Vibe_Ace.ogg


### Loading the file

This loads and decodes the audio as a time series y, represented as a one-dimensional NumPy floating point array. The variable sr contains the sampling rate of y, that is, the number of samples per second of audio. By default, all audio is mixed to mono and resampled to 22050 Hz at load time. This behavior can be overridden by supplying additional arguments to librosa.load().

In [12]:
# Load the audio as a waveform `y`
#    Store the sampling rate as `sr`
y, sr = librosa.load(filename)

print("sr: ", sr)
print("y - type: ", type(y))
print("y.shape: ", y.shape)
print("y: ", y)

sr:  22050
y - type:  <class 'numpy.ndarray'>
y.shape:  (1355168,)
y:  [0. 0. 0. ... 0. 0. 0.]


### Now we run the beat tracker

The output of the beat tracker is an estimate of the tempo (in beats per minute), and an array of frame numbers corresponding to detected beat events.

Frames here correspond to short windows of the signal (y), each separated by hop_length = 512 samples. Since v0.3, librosa uses centered frames, so that the kth frame is centered around sample k * hop_length.

In [13]:
# Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

Estimated tempo: 129.20 beats per minute


### Convert beat frames into timings

The next operation converts the frame numbers beat_frames into timings:
Now, beat_times will be an array of timestamps (in seconds) corresponding to detected beat events.


In [16]:
# Convert the frame indices of beat events into timestamps
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

print("type(beat_times): ", type(beat_times))
print("shape: ", beat_times.shape)
print("beat_times: ", beat_times)

type(beat_times):  <class 'numpy.ndarray'>
shape:  (129,)
beat_times:  [ 0.11609977  0.55727891  0.99845805  1.46285714  1.92725624  2.39165533
  2.83283447  3.29723356  3.76163265  4.22603175  4.69043084  5.15482993
  5.61922902  6.08362812  6.52480726  6.98920635  7.45360544  7.91800454
  8.38240363  8.87002268  9.31120181  9.77560091 10.24       10.70439909
 11.14557823 11.60997732 12.07437642 12.53877551 13.0031746  13.4675737
 13.93197279 14.39637188 14.83755102 15.27873016 15.74312925 16.20752834
 16.67192744 17.11310658 17.60072562 18.04190476 18.52952381 18.97070295
 19.43510204 19.89950113 20.36390023 20.80507937 21.29269841 21.73387755
 22.2214966  22.66267574 23.12707483 23.59147392 24.05587302 24.49705215
 24.96145125 25.42585034 25.91346939 26.35464853 26.81904762 27.28344671
 27.7478458  28.18902494 28.65342404 29.11782313 29.60544218 30.06984127
 30.53424036 30.9754195  31.43981859 31.88099773 32.36861678 32.83301587
 33.29741497 33.7385941  34.2029932  34.66739229 35.13

## Advanced usage

In [17]:
# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

### time-series harmonic-percussive separation:

The result of this line ("librosa.effects.hpss(y)") is that the time series y has been separated into two time series, containing the harmonic (tonal) and percussive (transient) portions of the signal. Each of y_harmonic and y_percussive have the same shape and duration as y.

The motivation for this kind of operation is two-fold: first, percussive elements tend to be stronger indicators of rhythmic content, and can help provide more stable beat tracking results; second, percussive elements can pollute tonal feature representations (such as chroma) by contributing energy across all frequency bands, so we’d be better off without them.

In [19]:
# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)
print("type(y_harmonic): ", type(y_harmonic), y_harmonic.shape)
print("type(y_percussive): ", type(y_percussive), y_percussive.shape)


type(y_harmonic):  <class 'numpy.ndarray'> (1355168,)
type(y_percussive):  <class 'numpy.ndarray'> (1355168,)


In [24]:
# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)


In [25]:
print("tempo: ", type(tempo), tempo.shape)
print("beat_frames: ", type(beat_frames), beat_frames.shape)

tempo:  <class 'numpy.float64'> ()
beat_frames:  <class 'numpy.ndarray'> (98,)


### Next, we introduce the feature module and extract the Mel-frequency cepstral coefficients from the raw signal y:

The output of this function is the matrix mfcc, which is an numpy.ndarray of size (n_mfcc, T) (where T denotes the track duration in frames). Note that we use the same hop_length here as in the beat tracker, so the detected beat_frames values correspond to columns of mfcc.

In [26]:
# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)



In [29]:
print(type(mfcc), mfcc.shape)
print(mfcc)

<class 'numpy.ndarray'> (13, 2647)
[[-522.94525   -494.4136    -402.84222   ... -522.94525   -522.94525
  -522.94525  ]
 [   0.          37.863792   125.671684  ...    0.           0.
     0.       ]
 [   0.          31.919342    48.412052  ...    0.           0.
     0.       ]
 ...
 [   0.           5.119666     7.4364023 ...    0.           0.
     0.       ]
 [   0.           1.9187412    5.807945  ...    0.           0.
     0.       ]
 [   0.           1.4670931    4.5749187 ...    0.           0.
     0.       ]]


### The first type of feature manipulation we introduce is delta, which computes (smoothed) first-order differences among columns of its input:

The resulting matrix mfcc_delta has the same shape as the input mfcc.

In [31]:
# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [34]:
print(type(mfcc_delta), mfcc_delta.shape)
print(mfcc_delta)

<class 'numpy.ndarray'> (13, 2647)
[[1.6657017e+01 1.6657017e+01 1.6657017e+01 ... 2.3879012e-14
  2.3879012e-14 2.3879012e-14]
 [1.7844133e+01 1.7844133e+01 1.7844133e+01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.6168957e+00 4.6168957e+00 4.6168957e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [9.9701935e-01 9.9701935e-01 9.9701935e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [7.1995008e-01 7.1995008e-01 7.1995008e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [7.6775694e-01 7.6775694e-01 7.6775694e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


### The second type of feature manipulation is sync, which aggregates columns of its input between sample indices (e.g., beat frames):

Here, we’ve vertically stacked the mfcc and mfcc_delta matrices together. The result of this operation is a matrix beat_mfcc_delta with the same number of rows as its input, but the number of columns depends on beat_frames. Each column beat_mfcc_delta[:, k] will be the average of input columns between beat_frames[k] and beat_frames[k+1]. (beat_frames will be expanded to span the full range [0, T] so that all data is accounted for.)

In [38]:
# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)



In [41]:
print(type(beat_mfcc_delta), beat_mfcc_delta.shape)
print(beat_mfcc_delta)

<class 'numpy.ndarray'> (26, 99)
[[-4.3199908e+02 -3.5066562e+02 -3.5409552e+02 ... -3.6887192e+02
  -3.6429688e+02 -4.6431967e+02]
 [ 7.6944107e+01  1.7167961e+02  1.7312457e+02 ...  1.5771146e+02
   1.6315385e+02  6.5702782e+01]
 [ 2.6619612e+01  4.8596348e+01  5.3320553e+01 ...  7.1283981e+01
   6.9528328e+01  3.7217251e+01]
 ...
 [ 9.9701935e-01 -3.6436152e-01 -1.0514270e-01 ... -3.2316583e-01
   4.4299937e-03  9.0835318e-02]
 [ 7.1995008e-01 -3.7288606e-01 -1.5928800e-01 ...  1.0751724e-01
  -3.3700153e-01  1.8190015e-02]
 [ 7.6775694e-01 -3.5967681e-01 -2.0632851e-01 ...  6.6612914e-02
  -5.4115180e-02 -2.2636700e-02]]


### Next, we compute a chromagram using just the harmonic component:

After this line, chromagram will be a numpy.ndarray of size (12, T), and each row corresponds to a pitch class (e.g., C, C#, etc.). Each column of chromagram is normalized by its peak value, though this behavior can be overridden by setting the norm parameter.

In [42]:
# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)



In [45]:
print("chromagram: ", type(chromagram), chromagram.shape)
print(chromagram)

chromagram:  <class 'numpy.ndarray'> (12, 2647)
[[0.39552355 0.35762173 0.26433796 ... 0.4369053  0.52066547 0.36136216]
 [0.7621055  0.3695345  0.17085999 ... 0.4669385  0.5907251  0.8903267 ]
 [0.82001483 0.6276381  0.3367408  ... 0.47863787 0.7427805  0.7165652 ]
 ...
 [0.05295175 0.13489355 0.08383726 ... 0.23072945 0.25645962 0.22334492]
 [0.28943363 0.22770779 0.1233625  ... 0.3557298  0.32827526 0.19588833]
 [0.37120998 0.29243177 0.21140918 ... 0.42624044 0.48065174 0.5643362 ]]


### Once we have the chromagram and list of beat frames, we again synchronize the chroma between beat events:

This time, we’ve replaced the default aggregate operation (average, as used above for MFCCs) with the median. In general, any statistical summarization function can be supplied here, including np.max(), np.min(), np.std(), etc.

In [46]:
# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)



In [49]:
print(type(beat_chroma), beat_chroma.shape)
print(beat_chroma)

<class 'numpy.ndarray'> (12, 99)
[[0.31097984 0.16985811 0.16698499 ... 0.10626511 0.27737483 0.174833  ]
 [0.27019724 0.09719466 0.12154159 ... 0.08623549 0.1457297  0.20282173]
 [0.48218945 0.14248455 0.16384622 ... 0.05574859 0.07817318 0.1622645 ]
 ...
 [0.07509185 0.03497266 0.03852925 ... 0.9967725  0.56940055 0.04644404]
 [0.17553514 0.13109866 0.12061698 ... 0.37875462 0.8188839  0.08942737]
 [0.25192046 0.23223288 0.22083756 ... 0.08240373 0.6653115  0.14264391]]


### Finally, the all features are vertically stacked again:

resulting in a feature matrix beat_features of dimension (12 + 13 + 13, # beat intervals).


In [51]:
# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [53]:
print(type(beat_features), beat_features.shape)
print(beat_features)

<class 'numpy.ndarray'> (38, 99)
[[ 0.31097984  0.16985811  0.16698499 ...  0.10626511  0.27737483
   0.174833  ]
 [ 0.27019724  0.09719466  0.12154159 ...  0.08623549  0.1457297
   0.20282173]
 [ 0.48218945  0.14248455  0.16384622 ...  0.05574859  0.07817318
   0.1622645 ]
 ...
 [ 0.99701935 -0.36436152 -0.1051427  ... -0.32316583  0.00442999
   0.09083532]
 [ 0.7199501  -0.37288606 -0.159288   ...  0.10751724 -0.33700153
   0.01819002]
 [ 0.76775694 -0.3596768  -0.20632851 ...  0.06661291 -0.05411518
  -0.0226367 ]]


## More examples

https://librosa.github.io/librosa/advanced.html#advanced
